In [ ]:
#!pip  install openai langchain langchain-community faiss-cpu langchain-openai tiktoken >/dev/null
!pip install -q langchain_openai==0.0.2 faiss-cpu==1.7.4 openai==1.6.1 tiktoken==0.5.2 langchain_community==0.0.11 langchain==0.1.0 python-telegram-bot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.7/618.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Imports
from google.colab import output
from google.colab import userdata

from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes, CallbackQueryHandler
from telegram import InlineKeyboardMarkup, Update, InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardMarkup
from telegram import Update

import openai
import os
import requests
import aiohttp
import json
import asyncio
import nest_asyncio

from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
#from langchain_openai import OpenAI
from openai import OpenAI
from langchain.chat_models import ChatOpenAI

import re
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from datetime import datetime

In [ ]:
#@title  Используем секретный ключ в колабе
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

In [ ]:
#@title функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
#@title База знаний, которая будет подаваться в langChain
#data_from_url= load_document_text('https://docs.google.com/document/d/1ova0gprJGQ7_wYJhxEozpzZTgsGYYgyADeNOm-l7rfM')
data_from_url = load_document_text('https://docs.google.com/document/d/1Tr7DfGNP9V1PcwTOhywHImT12h5krrmWapTTvyXyaDI') #Calendar
system_from_url = load_document_text('https://docs.google.com/document/d/1IESDe-22St33o8AKCIgDX2RLuSzILcOq8dxVUPWsjbU') #Jane

In [ ]:
#@title Разбиениe бд recursive cts

# source_chunks=[]
# splitter = RecursiveCharacterTextSplitter(chunk_size=2024, chunk_overlap=0)
# for chunk in splitter.split_text(data_from_url):
#     source_chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))
# len(source_chunks)

In [ ]:
#@title Разбиениe бд MarkDown
source_chunks=[]

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
source_chunks = markdown_splitter.split_text(data_from_url)
len(source_chunks)

2

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [ ]:
#@title определяем переменные и промпты
system_prompt = system_from_url
#model = "gpt-3.5-turbo-0125"
#model = "gpt-4-0125-preview"
model = "gpt-4-turbo-2024-04-09"
option = True
temperature = 0.4
num_fragment = 4
verbose = 0
user_prompt = "Information to answer user question, consider dialogue history and supplied information from the schedule."
user_prompt_2 = "You need to collect the information about desired meeting option from the user to setup the meeting with Mr Erik, consider dialogue history and supplied information from the schedule."
summary_memory = ConversationSummaryMemory(llm=ChatOpenAI(model_name=model))

In [ ]:
# @title sumarize old version
# def summarize_questions(dialog):
#     """
#     Функция возвращает саммаризированный текст диалога.
#     """
#     messages = [
#         {"role": "system", "content": "You are an AI summarizer, your task is to make a summary of a dialogue. Collect all relevant and important information from the text. If user mentions his name, add it to the summary"},
#         {"role": "user", "content": "Make a summary of this dialogue between users: " + " ".join(dialog)}
#     ]

#     completion = openai.ChatCompletion.create(
#        # model="gpt-4-0613",     # используем gpt4 для более точной саммаризации
#         model = "gpt-3.5-turbo-16k",
#         messages=messages,
#         temperature=0,          # Используем более низкую температуру для более определенной суммаризации
#     )
#     print('summary: '+completion.choices[0].message.content)
#     return completion.choices[0].message.content

# def answer_user_question_dialog(system, db, user_question, question_history):
#     """
#     Функция возвращает ответ на вопрос пользователя.
#     """
#     summarized_history = ""
#     # Если в истории более одного вопроса, применяем суммаризацию
#     if len(question_history) > 0:
#         summarized_history = "Previous dialogue: " + summarize_questions([q + ' ' + (a if a else '') for q, a in question_history])
#         #dialogue = ' '.join([q + ' ' + (a if a else '') for q, a in question_history])
#         #summarized_history = "Previous dialogue: " + summarize_questions(dialogue)

#     # Добавляем явное разделение между историей диалога и текущим вопросом
#     input_text = summarized_history + "\n\nCurrent user question: " + user_question

#     # Извлекаем наиболее похожие отрезки текста из базы знаний и получение ответа модели
#     answer_text = answer_index(system, input_text, db)

#     # Добавляем вопрос пользователя и ответ системы в историю
#     question_history.append((user_question, answer_text if answer_text else ''))

#     # Выводим саммаризированный текст, который видит модель
#     if summarized_history:
#         print('****************************')
#         print(insert_newlines(summarized_history))
#         print('****************************')

#     return insert_newlines(answer_text)

In [ ]:
#@title chat completion ex
# async def async_get_answer(self, system:str = default_system, query:str = None):
#       '''Асинхронная функция получения ответа от chatgpt
#       '''
#       # релевантные отрезки из базы
#       docs = self.db.similarity_search(query, k=4)
#       message_content = '\n'.join([f'{doc.page_content}' for doc in docs])
#       messages = [
#             {"role": "system", "content": system},
#             {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{query}"}
#       ]

#       # получение ответа от chatgpt
#       completion = await openai.ChatCompletion.acreate(model="gpt-3.5-turbo",
#                                                   messages=messages,
#                                                   temperature=0)

#       return completion.choices[0].message.content

In [ ]:
#@title Подготовка истории диалога

async def get_answer_async(text):
    #payload = {"text":text}
    payload = text

    # Загружаем историю сообщений из памяти
    history = summary_memory.load_memory_variables({})

    # Проверяем, является ли история списком сообщений или строкой
    if isinstance(history['history'], str):
          conversation_string_from_history = history['history']
    else:
        # Если история - это список сообщений, то формируем строку из их содержимого
        conversation_string_from_history = "\n".join(message.content for message in history['history'])
    print('conversation' + conversation_string_from_history)

  # модель должна ответить на вопрос в соответствии с промптом:
    if (option == False):
      user_prompt = user_prompt_2

    output = await answer_user_quest(system_prompt, payload, user_prompt, db, 0.2, verbose, num_fragment, model, hist=conversation_string_from_history)

  # Сохраняем вопрос пользователя и ответ модели в историю
    summary_memory.save_context({"input": payload}, {"output": output})

    return output


In [ ]:
#@title создаем функцию, которая будет получать ответ от модели на основании промптов, чанков и истории беседы с пользователем:
async def answer_user_quest(system, topic, instructions, search_index, temp, verbose, k, model, hist=''):
    client = OpenAI()
    docs = db.similarity_search_with_score(topic, k=k)
    message_content = '\n '.join([f'Отрывок текста №{i+1}\n{doc[0].page_content}' for i, doc in enumerate(docs)])

    messages = [{"role": "system", "content": system}, {"role": "user", "content": f"{instructions}\n\nInformation from calendar:\n{message_content}\n\n Chat history:\n{hist}\n\n user question:{topic}"}]

    completion = client.chat.completions.create(model=model, messages=messages, temperature=temp)

    return completion.choices[0].message.content

In [ ]:
async def text(update: Update, context: ContextTypes.DEFAULT_TYPE):

     # выполнение запроса в chatgpt
     first_message = await update.message.reply_text('Your request is being processed...')
     res = await get_answer_async(update.message.text)
     await context.bot.edit_message_text(text=res, chat_id=update.message.chat_id, message_id=first_message.message_id)


In [ ]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    # возвращаем текстовое сообщение пользователю
    #await update.message.reply_text('hello, start your conversation with assistant Jane')
    global option
    #option with buttons
    option = True
    await update.message.reply_text('Select what you want to do:', reply_markup=inline_keyboard)

async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE):
    message_ids = context.chat_data.get('message_ids', [])  # Get or default to empty list
    if message_ids:
        await context.bot.delete_messages(
            chat_id=update.effective_chat.id,
            message_ids=message_ids
        )
    context.chat_data['message_ids'] = []  # Clear regardless of whether it was empty
    summary_memory.clear()
    # Send restart message
    await update.message.reply_text('Conversation history cleared. Run /start to start again!')

async def stop(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Stopping bot...')
    application = context.application

    # Stop the application gracefully
    await application.stop()


    # # Get the current event loop
    # loop = asyncio.get_running_loop()

    # # Stop the loop gracefully
    # tasks = [t for t in asyncio.all_tasks() if t is not asyncio.current_task()]
    # for task in tasks:
    #     task.cancel()
    # await asyncio.gather(*tasks, return_exceptions=True)
    # loop.stop()




In [ ]:
# INLINE
# форма inline клавиатуры
inline_frame = [[InlineKeyboardButton("Schedule with Mr Erik", callback_data="with_er")],
                [InlineKeyboardButton("Ask to schedule for Mr Erik", callback_data="for_er")]]

# создаем inline клавиатуру
inline_keyboard = InlineKeyboardMarkup(inline_frame)

async def button(update: Update, _):
    global option
    # получаем callback query из update
    query = update.callback_query
    if query.data == "for_er":
      option = False
      await query.edit_message_text("My name is Jane, I'm mr Erik's assistant, he would like to setup a meeting with you, could you please share your availability the upcoming days")
    else:
      await query.edit_message_text("To Start your conversation with assistant Jane select what you want to do")
    # редактируем сообщение после нажатия



In [ ]:
nest_asyncio.apply()

def main():
    TOKEN = userdata.get('bot_token')
    # создаем приложение и передаем в него токен бота
    application = Application.builder().token(TOKEN).build()
    print('Бот запущен...')

    # добавление обработчиков
    application.add_handler(CommandHandler("stop", stop, block=False))
    application.add_handler(CommandHandler("clear", clear, block=False))
    application.add_handler(CommandHandler("start", start, block=False))
    application.add_handler(MessageHandler(filters.TEXT, text, block=False))
    # добавляем CallbackQueryHandler (только для inline кнопок)
    application.add_handler(CallbackQueryHandler(button))

    #запуск бота
    try:
        application.run_polling()
    except KeyboardInterrupt:
        print("Bot stopped manually.")
    finally:
        print("Exiting bot...")

if __name__ == "__main__":
    main()

Бот запущен...
conversation
conversationThe human expresses happiness about an upcoming meeting with Erik, available from April 18 to 26 after 2pm. The AI requests the human's name and additional details about the meeting, such as its length and format (Zoom or offline).
conversationThe human, named Din, expresses happiness about an upcoming meeting with Erik, available from April 18 to 26 after 2pm, and specifies it will be a 1-hour Zoom meeting. The AI requests Din to specify the exact date for the Zoom meeting within the given range.
conversationThe human, named Din, expresses happiness about an upcoming meeting with Erik, available from April 18 to 26 after 2pm, and specifies it will be a 1-hour Zoom meeting. When the AI requests Din to specify the exact date for the Zoom meeting within the given range, Din indicates flexibility, allowing the AI to choose. The AI then presents available time slots for each date within the range for Din to select the most suitable option.
conversati

RuntimeError: Cannot close a running event loop